# Tabulate phenotypes of SARS-CoV-2 spike that might have predictive power in understanding evolution for different mutations and clades

Import Python modules:

In [1]:
import json
import os
import urllib.request

import altair as alt

import numpy

import pandas as pd

import ruamel.yaml as yaml

Read the configuration YAML:

In [2]:
# This cell is tagged as parameters. So if you run the notebook using papermill
# (https://papermill.readthedocs.io/en/latest/usage-cli.html) with 
# `-p config_yaml <config_yaml_file>` then the file defined here will be replaced
# with the one you pass
config_yaml = "config.yaml"

In [3]:
with open(config_yaml) as f:
    config = yaml.YAML().load(f)

## Mutation effects on phenotypes
Read all the specified mutation-level phenotypic effects into a Data Frame.
Write that data frame to a file.
Also created data frame and file in which the mutation-level phenotypic data are randomized among mutations.

In [4]:
mutation_phenotypes = []
req_cols = ["site", "wildtype", "mutant"]
ref_clades = []
for ref_clade, ref_clade_d in config["mutation_phenotype_csvs"].items():
    ref_clades.append(ref_clade)
    for phenotype_category, category_d in ref_clade_d.items():
        pheno_cols = category_d["columns"]
        print(
            f"Reading {phenotype_category} phenotypes from {category_d['csv']}\n  "
            + "\n  ".join(pheno_cols)
        )
        if set(req_cols).intersection(pheno_cols):
            raise ValueError(f"{phenotype_category} columns {pheno_cols} include {req_cols}")
        df = pd.read_csv(category_d["csv"])
        if not set(req_cols + pheno_cols).issubset(df.columns):
            raise ValueError(f"cannot find expected columns for {phenotype_category}")
        mutation_phenotypes.append(
            df
            .melt(
                id_vars=req_cols,
                value_vars=pheno_cols,
                var_name="phenotype",
                value_name="mutation_effect",
            )
            .assign(ref_clade=ref_clade, phenotype_category=phenotype_category)
            [["ref_clade", "phenotype_category", "phenotype", *req_cols, "mutation_effect"]]
            .query("mutation_effect.notnull()")
            .query("wildtype != mutant")  # drop any wildtype to wildtype mutations
        )

mutation_phenotypes = pd.concat(mutation_phenotypes, ignore_index=True)

mutation_phenotypes_csv = config["mutation_phenotypes_csv"]
os.makedirs(os.path.dirname(mutation_phenotypes_csv), exist_ok=True)
print(f"\nWriting mutation phenotypes to {mutation_phenotypes_csv}")
mutation_phenotypes.to_csv(mutation_phenotypes_csv, index=False, float_format="%.4g")

n_randomizations = config["n_randomizations"]
print(f"\nNow randomizing each mutation phenotype {n_randomizations} times")
mutation_phenotypes_randomized = []
for _, df in mutation_phenotypes.groupby(["ref_clade", "phenotype_category", "phenotype"]):
    for random_seed in range(n_randomizations):
        numpy.random.seed(random_seed)
        mutation_phenotypes_randomized.append(
            df
            .assign(
                mutation_effect=lambda x: numpy.random.shuffle(x["mutation_effect"].to_numpy()),
                random_seed=random_seed
            )
            [["random_seed"] + df.columns.tolist()]
        )
mutation_phenotypes_randomized = pd.concat(mutation_phenotypes_randomized, ignore_index=True)
mutation_phenotypes_randomized_csv = config["mutation_phenotypes_randomized_csv"]
os.makedirs(os.path.dirname(mutation_phenotypes_randomized_csv), exist_ok=True)
print(f"Writing randomized mutation phenotypes to {mutation_phenotypes_randomized_csv}")
mutation_phenotypes_randomized.to_csv(mutation_phenotypes_randomized_csv, index=False, float_format="%.4g")

Reading spike pseudovirus DMS phenotypes from data/spike_pseudovirus_DMS_XBB.1.5.csv
  human sera escape
  spike mediated entry
  ACE2 binding
Reading RBD yeast-display DMS phenotypes from data/yeast_RBD_DMS_XBB.1.5.csv
  ACE2 affinity
  RBD expression
  escape
Reading EVEscape phenotypes from data/EVEscape_XBB_single_mutation_predictions.csv
  EVEscape
  fitness_evol_indices
  dissimilarity_charge_hydrophobicity
  accessibility_wcn

Writing mutation phenotypes to results/mutation_phenotypes.csv

Now randomizing each mutation phenotype 10 times
Writing randomized mutation phenotypes to results/mutation_phenotypes_randomized.csv


## Read the Pango clades
These come from Cornelius Roemer's GitHub repo ([https://github.com/corneliusroemer/pango-sequences](https://github.com/corneliusroemer/pango-sequences)):

In [5]:
pango_json = config["pango_json"]
print(f"Reading Pango clade definitions from {pango_json}")
with urllib.request.urlopen(pango_json) as url:
    pango_clades = json.load(url)
print(f"Read definitions for {len(pango_clades)} clades")

Reading Pango clade definitions from https://raw.githubusercontent.com/corneliusroemer/pango-sequences/main/data/pango-consensus-sequences_summary.json
Read definitions for 3810 clades


Get a data frame of all Pango clades along with other relevant information:

In [6]:
def is_descendant_of(clade, ancestor):
    """Returns True iff `clade` is a descendant of `ancestor`."""
    if pango_clades[clade]["parent"] == ancestor:
        return True
    elif pango_clades[clade]["parent"]:
        return is_descendant_of(pango_clades[clade]["parent"], ancestor)
    else:
        return False

def relative_mutations(clade_muts, reference_muts):
    """Get mutation in `clade_muts` relative `reference_muts`."""
    shared_muts = set(clade_muts).intersection(reference_muts)
    clade_sites = {
        r: (wt, m) for (wt, r, m) in [tup for tup in clade_muts if tup not in shared_muts]
    }
    reference_sites = {
        r: (wt, m) for (wt, r, m) in [tup for tup in reference_muts if tup not in shared_muts]
    }
    muts = []
    for r, (wt, m) in clade_sites.items():
        if r in reference_sites:
            assert wt == reference_sites[r][0]
            muts.append((r, reference_sites[r][1], m))
        else:
            muts.append((r, wt, m))
    for r, (wt, m) in reference_sites.items():
        if r in clade_sites:
            assert wt == clade_sites[r][0]
            pass  # already counted
        else:
            muts.append((r, m, wt))
    return [(wt, r, m) for (r, wt, m) in sorted(muts)]

def parse_spike_muts(clade_d):
    """Parse spike mutations from dict for a clade."""
    return [
        (mut.split(":")[1][0], int(mut.split(":")[1][1: -1]), mut.split(":")[1][-1])
        for mut in clade_d["aaSubstitutions"] + clade_d["aaDeletions"]
        if mut and mut.startswith("S:")
    ]

clades_df = {}

ref_clade_spike_muts = {
    ref_clade: parse_spike_muts(pango_clades[ref_clade])
    for ref_clade in ref_clades
}

for clade, clade_d in pango_clades.items():
    spike_muts = parse_spike_muts(clade_d)
    clades_df[clade] = {
        "date": clade_d["designationDate"] if clade_d["designationDate"] else pd.NA,
        "parent": clade_d["parent"] if clade_d["parent"] else pd.NA,
        **{
            f"spike_muts_relative_to_{ref_clade}":
                relative_mutations(spike_muts, ref_clade_spike_muts[ref_clade])
            for ref_clade in ref_clades
        },
        **{
            f"descendant_of_{ancestor}": is_descendant_of(clade, ancestor)
            for ancestor in config["classify_descendants_of"]
        },
        
    }
    
clades_df = pd.DataFrame.from_dict(clades_df, orient="index").reset_index(names="clade")

## Assign phenotypes to clades
Phenotypes are just sum of mutation effects of all mutations with respect to the `spike_muts_relative_to` clade.
Mutations without measured phenotype effects are ignored (given values of zero).

In [7]:
class PhenotypeAssigner:
    """Assign phenotypes to sets of mutations.

    Parameters
    ----------
    mutation_phenotypes_df : pandas.DataFrame
        Should have columns `site`, `wildtype`, `mutant`, `mutation_effect`.
        
    """
    def __init__(self, mutation_phenotypes_df):
        assert len(mutation_phenotypes_df) == len(
            mutation_phenotypes_df[["site", "mutant"]].drop_duplicates()
        )
        self.sites = sorted(set(mutation_phenotypes_df["site"]))
        assert len(self.sites) == len(
            mutation_phenotypes_df[["site", "wildtype"]].drop_duplicates()
        )
        self.wts = mutation_phenotypes_df.set_index("site")["wildtype"].to_dict()
        self.effects = {
            site: site_df.set_index("mutant")["mutation_effect"].to_dict()
            for site, site_df in mutation_phenotypes_df.groupby("site")
        }
        for site, wt in self.wts.items():
            assert wt not in self.effects[site]
            self.effects[site][wt] = 0.0

    def phenotype(self, muts):
        """Returns phenotype for list of `muts` as `(wildtype, site, mutant)`."""
        pheno = 0.0
        for wt, site, m in muts:
            if (site in self.effects) and (wt in self.effects[site]) and (m in self.effects[site]):
                pheno += self.effects[site][m] - self.effects[site][wt]
        return pheno

# assign clade phenotypes, Hamming distance from reference, and mutation lists as strings
clade_phenos_df = clades_df.copy()
for (ref_clade, phenotype_category, phenotype), df in mutation_phenotypes.groupby(
    ["ref_clade", "phenotype_category", "phenotype"]
):
    if (";" in ref_clade) or (";" in phenotype_category) or (";" in phenotype):
        raise ValueError(f"Cannot have ';' in {ref_clade=} or {phenotype_category=} or {phenotype=}")
    phenos = PhenotypeAssigner(df)
    clade_phenos_df[
        f"{ref_clade}; {phenotype_category}; {phenotype}"
    ] = clade_phenos_df[f"spike_muts_relative_to_{ref_clade}"].map(phenos.phenotype)
for ref_clade in ref_clades:
    col = f"spike_muts_relative_to_{ref_clade}"
    clade_phenos_df[f"{ref_clade}; Hamming distance; Hamming distance"] = clade_phenos_df[col].map(len)
    clade_phenos_df[col] = clade_phenos_df[col].map(
        lambda mutlist: " ".join(f"{wt}{r}{m}" for (wt, r, m) in mutlist)
    )

In [8]:
clade_phenos_df.columns

Index(['clade', 'date', 'parent', 'spike_muts_relative_to_XBB.1.5',
       'descendant_of_XBB', 'descendant_of_BA.2', 'descendant_of_BA.2.86',
       'XBB.1.5; EVEscape; EVEscape', 'XBB.1.5; EVEscape; accessibility_wcn',
       'XBB.1.5; EVEscape; dissimilarity_charge_hydrophobicity',
       'XBB.1.5; EVEscape; fitness_evol_indices',
       'XBB.1.5; RBD yeast-display DMS; ACE2 affinity',
       'XBB.1.5; RBD yeast-display DMS; RBD expression',
       'XBB.1.5; RBD yeast-display DMS; escape',
       'XBB.1.5; spike pseudovirus DMS; ACE2 binding',
       'XBB.1.5; spike pseudovirus DMS; human sera escape',
       'XBB.1.5; spike pseudovirus DMS; spike mediated entry',
       'XBB.1.5; Hamming distance; Hamming distance'],
      dtype='object')

In [9]:
" ".join(x for x in ["a", "b"])

'a b'